# **TCC - PERFIS DE DESEMPENHO NO ENEM(2018–2023) EM MUNICÍPIOS DA ÁREA DE ABRANGÊNCIA DO IFNMG**

Importando as bibliotecas que serão utilizadas.

In [10]:
import os

import pandas as pd
import numpy as np
import scipy.stats

import matplotlib.pyplot as plt

## **REPRESENTAÇÃO DE CONJUNTOS DE DADOS**
### **Leitura dos microdados do ENEM 2018**
- encoding='latin1' -> usado porque os arquivos do ENEM contêm acentos/caracteres especiais
- sep=";" -> separador oficial dos microdados do INEP é ponto e vírgula

In [11]:
df2018 = pd.read_csv("BASES\MICRODADOS_ENEM_2018.csv", encoding='latin1', sep=";")
df2018

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,Q026,Q027
0,180007186625,2018,2,F,0.0,4,1,2,0,4,...,B,C,B,B,D,B,C,B,B,D
1,180007186627,2018,3,F,0.0,4,1,1,1,1,...,A,B,A,A,B,A,A,A,A,A
2,180007186628,2018,3,M,0.0,0,1,2,0,2,...,A,B,B,A,D,A,A,A,B,A
3,180007186629,2018,2,F,0.0,3,1,2,0,4,...,A,C,A,B,D,B,A,B,B,B
4,180007186631,2018,11,F,0.0,5,1,1,6,1,...,A,B,B,A,C,A,B,A,A,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5513728,180014002713,2018,11,F,0.0,2,2,1,10,1,...,A,A,A,A,C,A,B,B,A,A
5513729,180014002715,2018,4,F,0.0,3,1,1,1,1,...,A,B,A,A,B,A,A,A,A,A
5513730,180014002716,2018,5,M,0.0,0,1,1,2,1,...,B,B,A,A,D,B,A,B,A,A
5513731,180014002721,2018,5,F,0.0,3,1,1,2,1,...,A,B,B,A,D,A,B,B,A,A


Informações gerais sobre o conjunto de dados
- verbose=True -> força a exibição de todas as colunas, mesmo quando o DataFrame é muito largo
- show_counts=True -> mostra a contagem de valores não nulos por coluna

In [12]:
df2018.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5513733 entries, 0 to 5513732
Data columns (total 78 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   NU_INSCRICAO            5513733 non-null  int64  
 1   NU_ANO                  5513733 non-null  int64  
 2   TP_FAIXA_ETARIA         5513733 non-null  int64  
 3   TP_SEXO                 5513733 non-null  object 
 4   TP_ESTADO_CIVIL         5296097 non-null  float64
 5   TP_COR_RACA             5513733 non-null  int64  
 6   TP_NACIONALIDADE        5513733 non-null  int64  
 7   TP_ST_CONCLUSAO         5513733 non-null  int64  
 8   TP_ANO_CONCLUIU         5513733 non-null  int64  
 9   TP_ESCOLA               5513733 non-null  int64  
 10  TP_ENSINO               3483075 non-null  float64
 11  IN_TREINEIRO            5513733 non-null  int64  
 12  CO_MUNICIPIO_ESC        1448816 non-null  float64
 13  NO_MUNICIPIO_ESC        1448816 non-null  object 
 14  CO

O conjunto de dados contém 5.513.733 observações (linhas) em 78 variáveis (colunas). Cada observação corresponde à inscrição de um participante do Exame Nacional do Ensino Médio (ENEM) realizado em 2018, enquanto as variáveis reúnem informações de natureza sociodemográfica, escolar, contexto socioeconômico, respostas aos questionários complementares e desempenho nas provas objetivas e de redação.

In [13]:
dicionario2018 = pd.read_excel(r"BASES\Dicionário_Microdados_Enem_2018.xlsx")
dicionario2018.columns = ["Column Name", "Description"]

# Exibir sem índice no Jupyter
from IPython.display import display
display(dicionario2018.style.hide(axis="index"))

Column Name,Description
NU_INSCRICAO,Número de inscrição
NU_ANO,Ano do Enem
TP_FAIXA_ETARIA,Faixa etária
TP_SEXO,Sexo
TP_ESTADO_CIVIL,Estado Civil
TP_COR_RACA,Cor/raça
TP_NACIONALIDADE,Nacionalidade
TP_ST_CONCLUSAO,Situação de conclusão do Ensino Médio
TP_ANO_CONCLUIU,Ano de Conclusão do Ensino Médio
TP_ESCOLA,Tipo de escola do Ensino Médio


A seguir, apresenta-se uma lista literal das variáveis do Questionário Socioeconômico do ENEM 2018 (Q001 a Q027), cujas respostas estão registradas nos microdados. Os esquemas de codificação para as respostas categóricas também são descritos (observe que, em variáveis binárias, os valores geralmente seguem a lógica de 1 para “Sim” e 0 para “Não”, enquanto nas perguntas com múltiplas categorias os códigos assumem valores ordinais definidos pelo INEP):

**TP_FAIXA_ETARIA**
- 1: Menor de 17 anos
- 2: 17 anos
- 3: 18 anos
- 4: 19 anos
- 5: 20 anos
- 6: 21 anos
- 7: 22 anos
- 8: 23 anos
- 9: 24 anos
- 10: 25 anos
- 11: Entre 26 e 30 anos
- 12: Entre 31 e 35 anos
- 13: Entre 36 e 40 anos
- 14: Entre 41 e 45 anos
- 15: Entre 46 e 50 anos
- 16: Entre 51 e 55 anos
- 17: Entre 56 e 60 anos
- 18: Entre 61 e 65 anos
- 19: Entre 66 e 70 anos
- 20: Maior de 70 anos

**TP_SEXO**
- M: Masculino
- F: Feminino

**TP_ESTADO_CIVIL**
- 0: Solteiro(a)
- 1: Casado(a) / Mora com companheiro(a)
- 2: Divorciado(a) / Desquitado(a) / Separado(a)
- 3: Viúvo(a)

**TP_COR_RACA**
- 0: Não declarado
- 1: Branca
- 2: Preta
- 3: Parda
- 4: Amarela
- 5: Indígena

**TP_NACIONALIDADE**
- 0: Não informado
- 1: Brasileiro(a)
- 2: Brasileiro(a) Naturalizado(a)
- 3: Estrangeiro(a)
- 4: Brasileiro(a) Nato(a), nascido(a) no exterior

**TP_ST_CONCLUSAO**
- 1: Já concluí o Ensino Médio
- 2: Estou cursando e concluirei o Ensino Médio em 2018
- 3: Estou cursando e concluirei o Ensino Médio após 2018
- 4: Não concluí e não estou cursando o Ensino Médio

**TP_ANO_CONCLUIU**
- 0: Não informado
- 1: 2017
- 2: 2016
- 3: 2015
- 4: 2014
- 5: 2013
- 6: 2012
- 7: 2011
- 8: 2010
- 9: 2009
- 10: 2008
- 11: 2007
- 12: Antes de 2007

**TP_ESCOLA**
- 1: Não respondeu
- 2: Pública
- 3: Exterior
- 4: Privada

**TP_ENSINO**
- 1: Ensino Regular
- 2: Educação Especial – Modalidade Substitutiva
- 3: Educação de Jovens e Adultos

**IN_TREINEIRO**
- 1: Sim
- 0: Não

**TP_DEPENDENCIA_ADM_ESC**  
- 1: Federal  
- 2: Estadual  
- 3: Municipal  
- 4: Privada  

**TP_LOCALIZACAO_ESC**  
- 1: Urbana  
- 2: Rural  

**TP_SIT_FUNC_ESC**  
- 1: Em atividade  
- 2: Paralisada  
- 3: Extinta  
- 4: Escola extinta em anos anteriores

**TP_PRESENCA_CN**  
- 0: Faltou à prova  
- 1: Presente na prova  
- 2: Eliminado na prova  

**TP_PRESENCA_CH**  
- 0: Faltou à prova  
- 1: Presente na prova  
- 2: Eliminado na prova  

**TP_PRESENCA_LC**  
- 0: Faltou à prova  
- 1: Presente na prova  
- 2: Eliminado na prova  

**TP_PRESENCA_MT**  
- 0: Faltou à prova  
- 1: Presente na prova  
- 2: Eliminado na prova  

**CO_PROVA_CN**  
- 447: Azul  
- 448: Amarela  
- 449: Cinza  
- 450: Rosa  
- 463: Laranja – Adaptada Ledor  
- 467: Verde – Videoprova – Libras  
- 487: Amarela (Reaplicação)  
- 488: Cinza (Reaplicação)  
- 489: Azul (Reaplicação)  
- 490: Rosa (Reaplicação)  

**CO_PROVA_CH**  
- 451: Azul  
- 452: Amarela  
- 453: Branca  
- 454: Rosa  
- 464: Laranja – Adaptada Ledor  
- 468: Verde – Videoprova – Libras  
- 491: Azul (Reaplicação)  
- 492: Amarela (Reaplicação)  
- 493: Branca (Reaplicação)  
- 494: Rosa (Reaplicação)  

**CO_PROVA_LC**  
- 455: Azul  
- 456: Amarela  
- 457: Rosa  
- 458: Branca  
- 465: Laranja – Adaptada Ledor  
- 469: Verde – Videoprova – Libras  
- 495: Azul (Reaplicação)  
- 496: Amarela (Reaplicação)  
- 497: Branca (Reaplicação)  
- 498: Rosa (Reaplicação)  

**CO_PROVA_MT**  
- 459: Azul  
- 460: Amarela  
- 461: Rosa  
- 462: Cinza  
- 466: Laranja – Adaptada Ledor  
- 470: Verde – Videoprova – Libras  
- 499: Amarela (Reaplicação)  
- 500: Cinza (Reaplicação)  
- 501: Azul (Reaplicação)  
- 502: Rosa (Reaplicação)

**TP_LINGUA – Língua Estrangeira**  
- 0: Inglês  
- 1: Espanhol  

**TP_STATUS_REDACAO**  
- 1: Sem problemas  
- 2: Anulada  
- 3: Cópia Texto Motivador  
- 4: Em Branco  
- 6: Fuga ao tema  
- 7: Não atendimento ao tipo textual  
- 8: Texto insuficiente  
- 9: Parte desconectada

**Q001 – Até que série seu pai, ou o homem responsável por você, estudou?**
- A: Nunca estudou.
- B: Não completou a 4ª série/5º ano do Ensino Fundamental.
- C: Completou a 4ª série/5º ano, mas não completou o 8º ano.
- D: Completou o Ensino Fundamental, mas não completou o Ensino Médio.
- E: Completou o Ensino Médio, mas não completou a Faculdade.
- F: Completou a Faculdade, mas não completou a Pós-graduação.
- G: Completou a Pós-graduação.

**Q002 – Até que série sua mãe, ou a mulher responsável por você, estudou?**
- A: Nunca estudou.
- B: Não completou a 4ª série/5º ano do Ensino Fundamental.
- C: Completou a 4ª série/5º ano, mas não completou o 8º ano.
- D: Completou o Ensino Fundamental, mas não completou o Ensino Médio.
- E: Completou o Ensino Médio, mas não completou a Faculdade.
- F: Completou a Faculdade, mas não completou a Pós-graduação.
- G: Completou a Pós-graduação.

**Q003 – A partir da apresentação de algumas ocupações divididas em grupos ordenados, indique o grupo que contempla a ocupação mais próxima da ocupação do seu pai ou do homem responsável por você. (Se ele não estiver trabalhando, escolha uma ocupação pensando no último trabalho dele).**
- A: Grupo 1 – Lavrador, agricultor sem empregados, bóia fria, criador de animais (gado, porcos, galinhas, ovelhas, cavalos etc.), apicultor, pescador, lenhador, seringueiro, extrativista.
- B: Grupo 2 – Diarista, empregado doméstico, cuidador de idosos, babá, cozinheiro (em casas particulares), motorista particular, jardineiro, faxineiro de empresas e prédios, vigilante, porteiro, carteiro, office-boy, vendedor, caixa, atendente de loja, auxiliar administrativo, recepcionista, servente de pedreiro, repositor de mercadoria.
- C: Grupo 3 – Padeiro, cozinheiro industrial ou em restaurantes, sapateiro, costureiro, joalheiro, torneiro mecânico, operador de máquinas, soldador, operário de fábrica, trabalhador da mineração, pedreiro, pintor, eletricista, encanador, motorista, caminhoneiro, taxista.
- D: Grupo 4 – Professor (fundamental/médio, idioma, música, artes etc.), técnico (enfermagem, contabilidade, eletrônica etc.), policial, militar de baixa patente (soldado, cabo, sargento), corretor de imóveis, supervisor, gerente, mestre de obras, pastor, microempresário (empresa <10 empregados), pequeno comerciante, pequeno proprietário de terras, trabalhador autônomo.
- E: Grupo 5 – Médico, engenheiro, dentista, psicólogo, economista, advogado, juiz, promotor, defensor, delegado, tenente, capitão, coronel, professor universitário, diretor em empresas públicas ou privadas, político, proprietário de empresas com mais de 10 empregados.
- F: Não sei.

**Q004 – A partir da apresentação de algumas ocupações divididas em grupos ordenados, indique o grupo que contempla a ocupação mais próxima da ocupação da sua mãe ou da mulher responsável por você. (Se ela não estiver trabalhando, escolha uma ocupação pensando no último trabalho dela).**
- A: Grupo 1 – Lavradora, agricultora sem empregados, bóia fria, criadora de animais, apicultora, pescadora, lenhadora, seringueira, extrativista.
- B: Grupo 2 – Diarista, empregada doméstica, cuidadora de idosos, babá, cozinheira (casas particulares), motorista particular, jardineira, faxineira, vigilante, porteira, carteira, office-boy, vendedora, caixa, atendente de loja, auxiliar administrativa, recepcionista, servente de pedreiro, repositora de mercadoria.
- C: Grupo 3 – Padeira, cozinheira industrial, sapateira, costureira, joalheira, torneira mecânica, operadora de máquinas, soldadora, operária de fábrica, mineradora, pedreira, pintora, eletricista, encanadora, motorista, caminhoneira, taxista.
- D: Grupo 4 – Professora, técnica, policial, militar de baixa patente, corretora de imóveis, supervisora, gerente, mestre de obras, pastora, microempresária (<10 empregados), pequena comerciante, pequena proprietária de terras, autônoma.
- E: Grupo 5 – Médica, engenheira, dentista, psicóloga, economista, advogada, juíza, promotora, defensora, delegada, tenente, capitã, coronel, professora universitária, diretora, política, proprietária de empresa >10 empregados.
- F: Não sei.

**Q005 – Incluindo você, quantas pessoas moram atualmente em sua residência?**
- 1: 1, pois moro sozinho(a).
- 2: 2
- 3: 3
- 4: 4
- 5: 5
- 6: 6
- 7: 7
- 8: 8
- 9: 9
- 10: 10
- 11: 11
- 12: 12
- 13: 13
- 14: 14
- 15: 15
- 16: 16
- 17: 17
- 18: 18
- 19: 19
- 20: 20

**Q006 – Qual é a renda mensal de sua família? (Some a sua renda com a dos seus familiares.)**

- A: Nenhuma renda
- B: Até R\$954,00.  
- C: De R\\$954,01 até R\\$1.431,00.  
- D: De R\\$1.431,01 até R\\$1.908,00.  
- E: De R\\$1.908,01 até R\\$2.385,00.  
- F: De R\\$2.385,01 até R\\$2.862,00.  
- G: De R\\$2.862,01 até R\\$3.816,00.  
- H: De R\\$3.816,01 até R\\$4.770,00.  
- I: De R\\$4.770,01 até R\\$5.724,00.  
- J: De R\\$5.724,01 até R\\$6.678,00.  
- K: De R\\$6.678,01 até R\\$7.632,00.  
- L: De R\\$7.632,01 até R\\$8.586,00.  
- M: De R\\$8.586,01 até R\\$9.540,00.  
- N: De R\\$9.540,01 até R\\$11.448,00.  
- O: De R\\$11.448,01 até R\\$14.310,00.  
- P: De R\\$14.310,01 até R\\$19.080,00. 
- Q: Mais de R\$19.080,00. 

**Q007 – Em sua residência trabalha empregado(a) doméstico(a)?**
- A: Não.
- B: Sim, um ou dois dias por semana.
- C: Sim, três ou quatro dias por semana.
- D: Sim, pelo menos cinco dias por semana.

**Q008 – Na sua residência tem banheiro?**
- A: Não.
- B: Sim, um.
- C: Sim, dois.
- D: Sim, três.
- E: Sim, quatro ou mais.

**Q009 – Na sua residência tem quartos para dormir?**
- A: Não.
- B: Sim, um.
- C: Sim, dois.
- D: Sim, três.
- E: Sim, quatro ou mais.

**Q010 – Na sua residência tem carro?**
- A: Não.
- B: Sim, um.
- C: Sim, dois.
- D: Sim, três.
- E: Sim, quatro ou mais.

**Q011 – Na sua residência tem motocicleta?**
- A: Não.
- B: Sim, uma.
- C: Sim, duas.
- D: Sim, três.
- E: Sim, quatro ou mais.

**Q012 – Na sua residência tem geladeira?**
- A: Não.
- B: Sim, uma.
- C: Sim, duas.
- D: Sim, três.
- E: Sim, quatro ou mais.

**Q013 – Na sua residência tem freezer (independente ou segunda porta da geladeira)?**
- A: Não.
- B: Sim, um.
- C: Sim, dois.
- D: Sim, três.
- E: Sim, quatro ou mais.

**Q014 – Na sua residência tem máquina de lavar roupa? (o tanquinho NÃO deve ser considerado)**
- A: Não.
- B: Sim, uma.
- C: Sim, duas.
- D: Sim, três.
- E: Sim, quatro ou mais.

**Q015 – Na sua residência tem máquina de secar roupa (independente ou em conjunto com a máquina de lavar roupa)?**
- A: Não.
- B: Sim, uma.
- C: Sim, duas.
- D: Sim, três.
- E: Sim, quatro ou mais.

**Q016 – Na sua residência tem forno micro-ondas?**
- A: Não.
- B: Sim, um.
- C: Sim, dois.
- D: Sim, três.
- E: Sim, quatro ou mais.

**Q017 – Na sua residência tem máquina de lavar louça?**
- A: Não.
- B: Sim, uma.
- C: Sim, duas.
- D: Sim, três.
- E: Sim, quatro ou mais.

**Q018 – Na sua residência tem aspirador de pó?**
- A: Não.
- B: Sim.

**Q019 – Na sua residência tem televisão em cores?**
- A: Não.
- B: Sim, uma.
- C: Sim, duas.
- D: Sim, três.
- E: Sim, quatro ou mais.

**Q020 – Na sua residência tem aparelho de DVD?**
- A: Não.
- B: Sim.

**Q021 – Na sua residência tem TV por assinatura?**
- A: Não.
- B: Sim.

**Q022 – Na sua residência tem telefone celular?**
- A: Não.
- B: Sim, um.
- C: Sim, dois.
- D: Sim, três.
- E: Sim, quatro ou mais.

**Q023 – Na sua residência tem telefone fixo?**
- A: Não.
- B: Sim.

**Q024 - Na sua residência tem computador?**
- A: Não.
- B: Sim, um.
- C: Sim, dois.
- D: Sim, três.
- E: Sim, quatro ou mais.

**Q025 – Na sua residência tem acesso à Internet?**
- A: Não
- B: Sim

**Q026 - Você já concluiu ou está concluindo o Ensino Médio?**
- A: Já concluí o Ensino Médio.
- B: Estou cursando e concluirei o Ensino Médio em 2018.
- C: Estou cursando e concluirei o Ensino Médio após 2018.
- D: Não concluí e não estou cursando o Ensino Médio.

**Q027 – Em que tipo de escola você frequentou o Ensino Médio?**
- A: Somente em escola pública
- B: A maior parte em escola pública
- C: Metade em escola pública e metade em escola privada
- D: A maior parte em escola privada
- E: Somente em escola privada

### **Variáveis excluídas da análise**

Embora o banco de dados original contenha um conjunto amplo de variáveis disponibilizadas pelo INEP, para este estudo optou-se por utilizar apenas aquelas consideradas mais relevantes para os objetivos da pesquisa. Assim, algumas variáveis foram excluídas, sendo elas:

Identificação do participante
- TP_ESTADO_CIVIL	
- TP_NACIONALIDADE	
- TP_ANO_CONCLUIU	
- IN_TREINEIRO

Dados do local de aplicação da prova
- CO_MUNICIPIO_PROVA
- NO_MUNICIPIO_PROVA
- CO_UF_PROVA
- SG_UF_PROVA

Dados da prova objetiva
- CO_PROVA_CN
- CO_PROVA_CH
- CO_PROVA_LC
- CO_PROVA_MT
- TX_RESPOSTAS_CN
- TX_RESPOSTAS_CH
- TX_RESPOSTAS_LC
- TX_RESPOSTAS_MT
- TX_GABARITO_CN
- TX_GABARITO_CH
- TX_GABARITO_LC
- TX_GABARITO_MT

Em resumo, optou-se pela exclusão de variáveis que, embora relevantes em outros contextos, não acrescentariam informações significativas para os objetivos deste estudo. As variáveis de identificação do participante (como estado civil, nacionalidade, ano de conclusão e condição de treineiro) indicam apenas aspectos sociodemográficos básicos. Já as variáveis relacionadas ao local de aplicação da prova (códigos e nomes de município e estado) servem apenas para mapear a logística do exame, sem impacto direto na análise proposta. Por fim, as variáveis referentes à prova objetiva (códigos das provas, respostas dos participantes e gabaritos oficiais) descrevem o processo avaliativo em si, mas foram desconsideradas por não estarem alinhadas ao foco desta pesquisa.

### **Leitura dos microdados do ENEM 2019**

In [14]:
df2019 = pd.read_csv("BASES\MICRODADOS_ENEM_2019.csv", encoding='latin1', sep=";")
df2019

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,190001595656,2019,13,M,1,3,1,1,11,1,...,A,A,A,A,A,A,C,A,C,B
1,190001421546,2019,8,M,1,1,1,1,5,1,...,A,A,A,B,A,A,E,A,A,B
2,190001133210,2019,13,F,1,3,1,1,13,1,...,A,A,A,B,A,A,C,A,A,B
3,190001199383,2019,10,F,1,1,1,1,4,1,...,A,A,A,C,A,B,D,A,B,B
4,190001237802,2019,7,F,1,1,1,1,4,1,...,B,A,A,B,A,A,C,B,B,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5095166,190006074437,2019,14,F,4,3,1,1,13,1,...,B,A,A,B,A,A,B,B,A,B
5095167,190005429225,2019,5,M,1,3,1,1,3,1,...,A,A,A,B,B,A,E,A,A,B
5095168,190006087652,2019,6,M,0,0,1,1,3,1,...,A,A,A,B,A,A,C,B,B,B
5095169,190006087653,2019,7,M,0,0,1,1,3,1,...,B,A,A,C,B,B,A,A,A,B


### **Leitura dos microdados do ENEM 2020**

In [15]:
df2020 = pd.read_csv("BASES\MICRODADOS_ENEM_2020.csv", encoding='latin1', sep=";")
df2020

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,200006271946,2020,11,F,1,2,1,1,11,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,200001195856,2020,11,M,2,3,1,1,11,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,200001943954,2020,4,F,2,3,2,2,0,2,...,B,A,A,B,A,A,A,A,A,A
3,200001908998,2020,2,M,1,3,1,2,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,200001634757,2020,4,F,1,3,2,1,1,1,...,A,A,A,B,A,B,B,A,A,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5783104,200003396891,2020,2,F,1,1,1,2,0,2,...,B,A,A,B,A,A,D,A,B,B
5783105,200006040822,2020,3,M,1,1,1,2,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5783106,200004928656,2020,3,F,1,1,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5783107,200006774808,2020,4,F,1,1,1,2,0,3,...,B,B,B,D,B,B,E,A,D,B


### **Leitura dos microdados do ENEM 2021**

In [16]:
df2021 = pd.read_csv("BASES\MICRODADOS_ENEM_2021.csv", encoding='latin1', sep=";")
df2021

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,210053865474,2021,5,F,1,1,1,1,3,1,...,A,A,A,B,A,A,B,A,B,B
1,210052384164,2021,12,M,1,1,1,1,11,1,...,A,A,A,B,A,A,C,A,A,A
2,210052589243,2021,13,F,3,1,1,1,15,1,...,B,A,A,B,A,A,C,B,B,B
3,210052128335,2021,3,M,1,3,1,2,0,2,...,A,A,A,B,A,A,B,A,B,B
4,210051353021,2021,2,F,1,3,1,2,0,2,...,B,A,A,B,A,B,E,A,B,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3389827,210053249138,2021,7,F,1,3,1,1,0,1,...,A,A,A,B,A,A,B,A,A,B
3389828,210053776013,2021,7,F,1,1,1,1,5,1,...,B,A,B,C,A,A,E,B,B,B
3389829,210052441508,2021,13,F,1,3,1,1,15,1,...,B,A,A,A,A,A,B,A,B,B
3389830,210051139675,2021,7,F,1,1,1,1,4,1,...,B,A,B,B,A,B,E,A,C,B


### **Leitura dos microdados do ENEM 2022**

In [17]:
df2022 = pd.read_csv("BASES\MICRODADOS_ENEM_2022.csv", encoding='latin1', sep=";")
df2022

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,210057943671,2022,14,M,2,2,1,1,2,1,...,B,A,A,A,A,A,A,A,A,A
1,210057516120,2022,14,M,2,1,1,1,16,1,...,E,E,B,E,B,B,E,B,E,B
2,210057280536,2022,5,F,1,2,1,1,2,1,...,A,A,A,A,A,A,C,A,A,B
3,210055724397,2022,6,M,1,3,1,1,2,1,...,B,A,A,C,A,A,C,B,B,B
4,210055097896,2022,4,M,0,3,1,1,1,1,...,A,A,A,A,A,A,B,A,A,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3476100,210056389903,2022,3,M,1,1,1,2,0,2,...,B,A,B,B,B,A,E,A,A,B
3476101,210057205644,2022,14,F,2,1,1,2,0,2,...,A,A,A,B,A,A,C,A,B,B
3476102,210056699189,2022,2,M,1,1,1,2,0,2,...,B,A,B,C,B,A,E,A,B,B
3476103,210056983033,2022,3,M,1,3,1,2,0,3,...,B,A,B,D,A,A,E,A,B,B


### **Leitura dos microdados do ENEM 2023**

In [18]:
df2023 = pd.read_csv("BASES\MICRODADOS_ENEM_2023.csv", encoding='latin1', sep=";")
df2023

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,210059085136,2023,14,M,2,1,1,1,17,1,...,C,C,B,B,A,B,B,A,A,B
1,210059527735,2023,12,M,2,1,0,1,16,1,...,B,A,B,B,A,A,C,A,D,B
2,210061103945,2023,6,F,1,1,1,1,0,1,...,B,A,A,B,A,A,A,A,A,B
3,210060214087,2023,2,F,1,3,1,2,0,2,...,A,A,A,B,A,A,D,A,A,B
4,210059980948,2023,3,F,1,3,1,2,0,2,...,A,A,A,B,A,A,B,A,A,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3933950,210061959676,2023,12,M,1,1,1,1,6,1,...,B,A,A,C,A,B,E,A,A,B
3933951,210061950911,2023,1,F,1,1,2,3,0,1,...,B,A,B,C,B,B,B,B,C,B
3933952,210061965966,2023,3,F,1,3,1,2,0,2,...,A,A,A,B,A,A,B,A,A,B
3933953,210061932304,2023,2,M,1,1,1,2,0,3,...,B,B,B,C,A,A,D,A,C,B


## **PRÉ-PROCESSAMENTO DE DADOS**
Esta seção apresenta as etapas de pré-processamento aplicadas aos dados antes da análise. São descritos os critérios adotados para a seleção, fusão e limpeza dos conjuntos de dados.

### **1. Remoção de colunas não utilizadas e filtro das cidades**

Devido ao grande volume dos microdados do ENEM, o tratamento foi realizado separadamente para cada ano. Inicialmente, foram removidas as colunas que não seriam utilizadas na pesquisa. Em seguida, os dados foram filtrados pela coluna **NO_MUNICIPIO_ESC**, de modo a considerar apenas os municípios pertencentes à área de abrangência do IFNMG. Esse procedimento reduz o tamanho da base e facilita a etapa posterior de consolidação em um único dataset.

**Remoção e filtro dos microdados do ENEM 2018**

In [19]:
cidades_filtro = [
    "Almenara","Alvorada de Minas","Angelândia","Araçuaí","Aricanduva","Arinos","Ataléia",
    "Augusto de Lima","Bandeira","Berilo","Berizal","Bertópolis","Bocaiúva","Bonfinópolis de Minas",
    "Bonito de Minas","Botumirim","Brasília de Minas","Buenópolis","Buritis","Buritizeiro","Cabeceira Grande",
    "Cachoeira de Pajeú","Campanário","Campo Azul","Capelinha","Capitão Enéas","Caraí","Carbonita",
    "Carlos Chagas","Catuji","Catuti","Chapada Gaúcha","Chapada do Norte","Claro dos Poções","Comercinho",
    "Coração de Jesus","Corinto","Coronel Murta","Couto de Magalhães de Minas","Cristália","Crisólita",
    "Curral de Dentro","Curvelo","Cônego Marinho","Datas","Diamantina","Divisa Alegre","Divisópolis",
    "Dom Bosco","Engenheiro Navarro","Espinosa","Felisburgo","Felixlândia","Felício dos Santos","Formoso",
    "Francisco Badaró","Francisco Dumont","Francisco Sá","Franciscópolis","Frei Gaspar","Fronteira dos Vales",
    "Fruta de Leite","Gameleiras","Glaucilândia","Gouveia","Grão Mogol","Guaraciama","Ibiaí","Ibiracatu",
    "Icaraí de Minas","Indaiabira","Inimutaba","Itacambira","Itacarambi","Itaipé","Itamarandiba","Itambacuri",
    "Itaobim","Itinga","Jacinto","Janaúba","Januária","Japonvar","Jaíba","Jenipapo de Minas","Jequitaí",
    "Jequitinhonha","Joaquim Felício","Joaíma","Jordânia","Josenópolis","José Gonçalves de Minas","Juramento",
    "Juvenília","Ladainha","Lagoa dos Patos","Lassance","Leme do Prado","Lontra","Luislândia","Machacalis",
    "Malacacheta","Mamonas","Manga","Mata Verde","Matias Cardoso","Mato Verde","Medina","Minas Novas",
    "Mirabela","Miravânia","Monjolos","Montalvânia","Monte Azul","Monte Formoso","Montes Claros","Montezuma",
    "Morro da Garça","Nanuque","Natalândia","Ninheira","Nova Módica","Nova Porteirinha","Novo Cruzeiro",
    "Novo Oriente de Minas","Novorizonte","Olhos-d'Água","Ouro Verde de Minas","Padre Carvalho","Padre Paraíso",
    "Pai Pedro","Palmópolis","Patis","Pavão","Pedra Azul","Pedras de Maria da Cruz","Pescador","Pintópolis",
    "Pirapora","Ponto Chique","Ponto dos Volantes","Porteirinha","Poté","Presidente Juscelino",
    "Presidente Kubitschek","Riachinho","Riacho dos Machados","Rio Pardo de Minas","Rio do Prado","Rubelita",
    "Rubim","Salinas","Salto da Divisa","Santa Cruz de Salinas","Santa Fé de Minas","Santa Helena de Minas",
    "Santa Maria do Salto","Santo Antônio do Itambé","Santo Antônio do Jacinto","Santo Antônio do Retiro",
    "Santo Hipólito","Senador Modestino Gonçalves","Serra Azul de Minas","Serra dos Aimorés","Serranópolis de Minas",
    "Serro","Setubinha","São Francisco","São Gonçalo do Rio Preto","São José do Divino","São João da Lagoa",
    "São João da Ponte","São João das Missões","São João do Pacuí","São João do Paraíso","São Romão","Taiobeiras",
    "Teófilo Otoni","Três Marias","Turmalina","Ubaí","Umburatiba","Unaí","Uruana de Minas","Urucuia",
    "Vargem Grande do Rio Pardo","Varzelândia","Verdelândia","Veredinha","Virgem da Lapa","Várzea da Palma",
    "Água Boa","Águas Formosas","Águas Vermelhas"
]


df2018_reduzida = df2018.drop(columns=[
    'TP_ESTADO_CIVIL','TP_NACIONALIDADE','TP_ANO_CONCLUIU','IN_TREINEIRO',
    'CO_MUNICIPIO_PROVA','NO_MUNICIPIO_PROVA','CO_UF_PROVA','SG_UF_PROVA',
    'CO_PROVA_CN','CO_PROVA_CH','CO_PROVA_LC','CO_PROVA_MT',
    'TX_RESPOSTAS_CN','TX_RESPOSTAS_CH','TX_RESPOSTAS_LC','TX_RESPOSTAS_MT',
    'TX_GABARITO_CN','TX_GABARITO_CH','TX_GABARITO_LC','TX_GABARITO_MT',
    'Q026','Q027'
])
df2018_reduzida = df2018_reduzida[df2018_reduzida['NO_MUNICIPIO_ESC'].isin(cidades_filtro)]
df2018_reduzida

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ESCOLA,TP_ENSINO,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
2,180007186628,2018,3,M,0,2,2,1.0,3155603.0,Rio Pardo de Minas,...,A,A,A,B,B,A,D,A,A,A
5,180007186632,2018,2,F,3,2,2,1.0,3161106.0,São Francisco,...,A,A,A,B,B,A,D,A,A,B
132,180007186772,2018,2,M,3,2,2,1.0,3142700.0,Montalvânia,...,A,A,A,B,A,A,C,A,B,B
224,180007186871,2018,2,M,3,2,2,1.0,3116100.0,Chapada do Norte,...,B,A,A,C,B,A,C,A,B,B
289,180007186944,2018,3,F,1,2,2,1.0,3168606.0,Teófilo Otoni,...,B,A,A,B,A,B,E,A,B,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5511073,180013997544,2018,4,F,3,2,2,3.0,1100452.0,Buritis,...,A,A,A,A,A,A,B,A,B,B
5511783,180013998872,2018,15,M,1,2,2,3.0,1100452.0,Buritis,...,A,A,A,B,A,A,B,A,B,B
5512025,180013999422,2018,3,F,3,2,2,1.0,3170404.0,Unaí,...,A,A,A,B,A,A,A,A,A,A
5512609,180014000579,2018,3,M,3,2,2,1.0,3118809.0,Coração de Jesus,...,A,A,A,B,A,A,C,A,A,A


**Remoção e filtro dos microdados do ENEM 2019**

In [20]:
cidades_filtro = [
    "Almenara","Alvorada de Minas","Angelândia","Araçuaí","Aricanduva","Arinos","Ataléia",
    "Augusto de Lima","Bandeira","Berilo","Berizal","Bertópolis","Bocaiúva","Bonfinópolis de Minas",
    "Bonito de Minas","Botumirim","Brasília de Minas","Buenópolis","Buritis","Buritizeiro","Cabeceira Grande",
    "Cachoeira de Pajeú","Campanário","Campo Azul","Capelinha","Capitão Enéas","Caraí","Carbonita",
    "Carlos Chagas","Catuji","Catuti","Chapada Gaúcha","Chapada do Norte","Claro dos Poções","Comercinho",
    "Coração de Jesus","Corinto","Coronel Murta","Couto de Magalhães de Minas","Cristália","Crisólita",
    "Curral de Dentro","Curvelo","Cônego Marinho","Datas","Diamantina","Divisa Alegre","Divisópolis",
    "Dom Bosco","Engenheiro Navarro","Espinosa","Felisburgo","Felixlândia","Felício dos Santos","Formoso",
    "Francisco Badaró","Francisco Dumont","Francisco Sá","Franciscópolis","Frei Gaspar","Fronteira dos Vales",
    "Fruta de Leite","Gameleiras","Glaucilândia","Gouveia","Grão Mogol","Guaraciama","Ibiaí","Ibiracatu",
    "Icaraí de Minas","Indaiabira","Inimutaba","Itacambira","Itacarambi","Itaipé","Itamarandiba","Itambacuri",
    "Itaobim","Itinga","Jacinto","Janaúba","Januária","Japonvar","Jaíba","Jenipapo de Minas","Jequitaí",
    "Jequitinhonha","Joaquim Felício","Joaíma","Jordânia","Josenópolis","José Gonçalves de Minas","Juramento",
    "Juvenília","Ladainha","Lagoa dos Patos","Lassance","Leme do Prado","Lontra","Luislândia","Machacalis",
    "Malacacheta","Mamonas","Manga","Mata Verde","Matias Cardoso","Mato Verde","Medina","Minas Novas",
    "Mirabela","Miravânia","Monjolos","Montalvânia","Monte Azul","Monte Formoso","Montes Claros","Montezuma",
    "Morro da Garça","Nanuque","Natalândia","Ninheira","Nova Módica","Nova Porteirinha","Novo Cruzeiro",
    "Novo Oriente de Minas","Novorizonte","Olhos-d'Água","Ouro Verde de Minas","Padre Carvalho","Padre Paraíso",
    "Pai Pedro","Palmópolis","Patis","Pavão","Pedra Azul","Pedras de Maria da Cruz","Pescador","Pintópolis",
    "Pirapora","Ponto Chique","Ponto dos Volantes","Porteirinha","Poté","Presidente Juscelino",
    "Presidente Kubitschek","Riachinho","Riacho dos Machados","Rio Pardo de Minas","Rio do Prado","Rubelita",
    "Rubim","Salinas","Salto da Divisa","Santa Cruz de Salinas","Santa Fé de Minas","Santa Helena de Minas",
    "Santa Maria do Salto","Santo Antônio do Itambé","Santo Antônio do Jacinto","Santo Antônio do Retiro",
    "Santo Hipólito","Senador Modestino Gonçalves","Serra Azul de Minas","Serra dos Aimorés","Serranópolis de Minas",
    "Serro","Setubinha","São Francisco","São Gonçalo do Rio Preto","São José do Divino","São João da Lagoa",
    "São João da Ponte","São João das Missões","São João do Pacuí","São João do Paraíso","São Romão","Taiobeiras",
    "Teófilo Otoni","Três Marias","Turmalina","Ubaí","Umburatiba","Unaí","Uruana de Minas","Urucuia",
    "Vargem Grande do Rio Pardo","Varzelândia","Verdelândia","Veredinha","Virgem da Lapa","Várzea da Palma",
    "Água Boa","Águas Formosas","Águas Vermelhas"
]
df2019_reduzida = df2019.drop(columns=[
    'TP_ESTADO_CIVIL','TP_NACIONALIDADE','TP_ANO_CONCLUIU','IN_TREINEIRO',
    'CO_MUNICIPIO_PROVA','NO_MUNICIPIO_PROVA','CO_UF_PROVA','SG_UF_PROVA',
    'CO_PROVA_CN','CO_PROVA_CH','CO_PROVA_LC','CO_PROVA_MT',
    'TX_RESPOSTAS_CN','TX_RESPOSTAS_CH','TX_RESPOSTAS_LC','TX_RESPOSTAS_MT',
    'TX_GABARITO_CN','TX_GABARITO_CH','TX_GABARITO_LC','TX_GABARITO_MT'])

df2019_reduzida = df2019_reduzida[df2019_reduzida['NO_MUNICIPIO_ESC'].isin(cidades_filtro)]
df2019_reduzida

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ESCOLA,TP_ENSINO,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
754,190001241391,2019,10,M,3,2,2,NaN,3170404.0,Unaí,...,A,A,A,B,A,A,B,A,B,B
1615,190001255713,2019,2,F,2,2,2,1.0,3135100.0,Janaúba,...,B,A,A,B,A,B,D,A,B,B
2943,190001248571,2019,2,F,3,2,2,1.0,3131109.0,Inimutaba,...,B,A,A,B,A,A,D,A,B,B
4501,190001410700,2019,2,M,3,2,2,1.0,3142700.0,Montalvânia,...,B,A,A,B,A,A,C,A,B,B
5146,190001252214,2019,3,M,3,2,2,1.0,3132107.0,Itacarambi,...,A,A,A,B,A,A,C,A,A,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5085427,190005492127,2019,3,M,3,2,2,NaN,3143302.0,Montes Claros,...,A,A,A,B,A,A,D,A,B,B
5087833,190005795089,2019,2,M,3,2,2,1.0,3170404.0,Unaí,...,B,A,A,B,A,A,D,A,A,B
5088681,190005766764,2019,4,M,2,2,2,1.0,3161106.0,São Francisco,...,A,A,A,B,A,B,C,A,A,A
5089133,190005794864,2019,2,M,2,2,2,1.0,3103405.0,Araçuaí,...,A,A,A,B,A,A,E,A,B,B


**Remoção e filtro dos microdados do ENEM 2020**

In [21]:
cidades_filtro = [
    "Almenara","Alvorada de Minas","Angelândia","Araçuaí","Aricanduva","Arinos","Ataléia",
    "Augusto de Lima","Bandeira","Berilo","Berizal","Bertópolis","Bocaiúva","Bonfinópolis de Minas",
    "Bonito de Minas","Botumirim","Brasília de Minas","Buenópolis","Buritis","Buritizeiro","Cabeceira Grande",
    "Cachoeira de Pajeú","Campanário","Campo Azul","Capelinha","Capitão Enéas","Caraí","Carbonita",
    "Carlos Chagas","Catuji","Catuti","Chapada Gaúcha","Chapada do Norte","Claro dos Poções","Comercinho",
    "Coração de Jesus","Corinto","Coronel Murta","Couto de Magalhães de Minas","Cristália","Crisólita",
    "Curral de Dentro","Curvelo","Cônego Marinho","Datas","Diamantina","Divisa Alegre","Divisópolis",
    "Dom Bosco","Engenheiro Navarro","Espinosa","Felisburgo","Felixlândia","Felício dos Santos","Formoso",
    "Francisco Badaró","Francisco Dumont","Francisco Sá","Franciscópolis","Frei Gaspar","Fronteira dos Vales",
    "Fruta de Leite","Gameleiras","Glaucilândia","Gouveia","Grão Mogol","Guaraciama","Ibiaí","Ibiracatu",
    "Icaraí de Minas","Indaiabira","Inimutaba","Itacambira","Itacarambi","Itaipé","Itamarandiba","Itambacuri",
    "Itaobim","Itinga","Jacinto","Janaúba","Januária","Japonvar","Jaíba","Jenipapo de Minas","Jequitaí",
    "Jequitinhonha","Joaquim Felício","Joaíma","Jordânia","Josenópolis","José Gonçalves de Minas","Juramento",
    "Juvenília","Ladainha","Lagoa dos Patos","Lassance","Leme do Prado","Lontra","Luislândia","Machacalis",
    "Malacacheta","Mamonas","Manga","Mata Verde","Matias Cardoso","Mato Verde","Medina","Minas Novas",
    "Mirabela","Miravânia","Monjolos","Montalvânia","Monte Azul","Monte Formoso","Montes Claros","Montezuma",
    "Morro da Garça","Nanuque","Natalândia","Ninheira","Nova Módica","Nova Porteirinha","Novo Cruzeiro",
    "Novo Oriente de Minas","Novorizonte","Olhos-d'Água","Ouro Verde de Minas","Padre Carvalho","Padre Paraíso",
    "Pai Pedro","Palmópolis","Patis","Pavão","Pedra Azul","Pedras de Maria da Cruz","Pescador","Pintópolis",
    "Pirapora","Ponto Chique","Ponto dos Volantes","Porteirinha","Poté","Presidente Juscelino",
    "Presidente Kubitschek","Riachinho","Riacho dos Machados","Rio Pardo de Minas","Rio do Prado","Rubelita",
    "Rubim","Salinas","Salto da Divisa","Santa Cruz de Salinas","Santa Fé de Minas","Santa Helena de Minas",
    "Santa Maria do Salto","Santo Antônio do Itambé","Santo Antônio do Jacinto","Santo Antônio do Retiro",
    "Santo Hipólito","Senador Modestino Gonçalves","Serra Azul de Minas","Serra dos Aimorés","Serranópolis de Minas",
    "Serro","Setubinha","São Francisco","São Gonçalo do Rio Preto","São José do Divino","São João da Lagoa",
    "São João da Ponte","São João das Missões","São João do Pacuí","São João do Paraíso","São Romão","Taiobeiras",
    "Teófilo Otoni","Três Marias","Turmalina","Ubaí","Umburatiba","Unaí","Uruana de Minas","Urucuia",
    "Vargem Grande do Rio Pardo","Varzelândia","Verdelândia","Veredinha","Virgem da Lapa","Várzea da Palma",
    "Água Boa","Águas Formosas","Águas Vermelhas"
]
df2020_reduzida = df2020.drop(columns=['TP_ESTADO_CIVIL','TP_NACIONALIDADE','TP_ANO_CONCLUIU','IN_TREINEIRO',
    'CO_MUNICIPIO_PROVA','NO_MUNICIPIO_PROVA','CO_UF_PROVA','SG_UF_PROVA',
    'CO_PROVA_CN','CO_PROVA_CH','CO_PROVA_LC','CO_PROVA_MT',
    'TX_RESPOSTAS_CN','TX_RESPOSTAS_CH','TX_RESPOSTAS_LC','TX_RESPOSTAS_MT',
    'TX_GABARITO_CN','TX_GABARITO_CH','TX_GABARITO_LC','TX_GABARITO_MT'])

df2020_reduzida = df2020_reduzida[df2020_reduzida['NO_MUNICIPIO_ESC'].isin(cidades_filtro)]
df2020_reduzida

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ESCOLA,TP_ENSINO,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
406,200006675118,2020,4,F,2,2,2,1.0,3143302.0,Montes Claros,...,B,A,A,B,A,A,C,A,B,B
1072,200001328818,2020,2,M,3,2,2,1.0,3143302.0,Montes Claros,...,C,A,A,C,A,A,D,A,B,B
1336,200004824060,2020,5,M,3,2,2,1.0,3166956.0,Serranópolis de Minas,...,A,A,A,B,A,A,C,A,A,B
1524,200003839970,2020,2,F,3,2,2,1.0,3103405.0,Araçuaí,...,B,A,A,B,A,A,C,A,B,B
4112,200006102683,2020,3,F,2,2,2,1.0,3168606.0,Teófilo Otoni,...,A,A,A,A,A,B,C,A,B,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5767119,200006203371,2020,3,F,3,2,2,1.0,3151206.0,Pirapora,...,A,A,A,B,A,A,D,A,B,B
5769232,200005714909,2020,4,M,1,2,2,1.0,3119500.0,Coronel Murta,...,A,A,A,B,A,A,B,A,A,B
5778650,200001250994,2020,3,M,3,2,2,1.0,3130051.0,Icaraí de Minas,...,A,A,A,B,A,B,E,A,A,B
5779115,200003528331,2020,6,M,1,2,2,1.0,3170404.0,Unaí,...,A,A,A,B,B,A,C,A,B,B


**Remoção e filtro dos microdados do ENEM 2021**

In [22]:
cidades_filtro = [
    "Almenara","Alvorada de Minas","Angelândia","Araçuaí","Aricanduva","Arinos","Ataléia",
    "Augusto de Lima","Bandeira","Berilo","Berizal","Bertópolis","Bocaiúva","Bonfinópolis de Minas",
    "Bonito de Minas","Botumirim","Brasília de Minas","Buenópolis","Buritis","Buritizeiro","Cabeceira Grande",
    "Cachoeira de Pajeú","Campanário","Campo Azul","Capelinha","Capitão Enéas","Caraí","Carbonita",
    "Carlos Chagas","Catuji","Catuti","Chapada Gaúcha","Chapada do Norte","Claro dos Poções","Comercinho",
    "Coração de Jesus","Corinto","Coronel Murta","Couto de Magalhães de Minas","Cristália","Crisólita",
    "Curral de Dentro","Curvelo","Cônego Marinho","Datas","Diamantina","Divisa Alegre","Divisópolis",
    "Dom Bosco","Engenheiro Navarro","Espinosa","Felisburgo","Felixlândia","Felício dos Santos","Formoso",
    "Francisco Badaró","Francisco Dumont","Francisco Sá","Franciscópolis","Frei Gaspar","Fronteira dos Vales",
    "Fruta de Leite","Gameleiras","Glaucilândia","Gouveia","Grão Mogol","Guaraciama","Ibiaí","Ibiracatu",
    "Icaraí de Minas","Indaiabira","Inimutaba","Itacambira","Itacarambi","Itaipé","Itamarandiba","Itambacuri",
    "Itaobim","Itinga","Jacinto","Janaúba","Januária","Japonvar","Jaíba","Jenipapo de Minas","Jequitaí",
    "Jequitinhonha","Joaquim Felício","Joaíma","Jordânia","Josenópolis","José Gonçalves de Minas","Juramento",
    "Juvenília","Ladainha","Lagoa dos Patos","Lassance","Leme do Prado","Lontra","Luislândia","Machacalis",
    "Malacacheta","Mamonas","Manga","Mata Verde","Matias Cardoso","Mato Verde","Medina","Minas Novas",
    "Mirabela","Miravânia","Monjolos","Montalvânia","Monte Azul","Monte Formoso","Montes Claros","Montezuma",
    "Morro da Garça","Nanuque","Natalândia","Ninheira","Nova Módica","Nova Porteirinha","Novo Cruzeiro",
    "Novo Oriente de Minas","Novorizonte","Olhos-d'Água","Ouro Verde de Minas","Padre Carvalho","Padre Paraíso",
    "Pai Pedro","Palmópolis","Patis","Pavão","Pedra Azul","Pedras de Maria da Cruz","Pescador","Pintópolis",
    "Pirapora","Ponto Chique","Ponto dos Volantes","Porteirinha","Poté","Presidente Juscelino",
    "Presidente Kubitschek","Riachinho","Riacho dos Machados","Rio Pardo de Minas","Rio do Prado","Rubelita",
    "Rubim","Salinas","Salto da Divisa","Santa Cruz de Salinas","Santa Fé de Minas","Santa Helena de Minas",
    "Santa Maria do Salto","Santo Antônio do Itambé","Santo Antônio do Jacinto","Santo Antônio do Retiro",
    "Santo Hipólito","Senador Modestino Gonçalves","Serra Azul de Minas","Serra dos Aimorés","Serranópolis de Minas",
    "Serro","Setubinha","São Francisco","São Gonçalo do Rio Preto","São José do Divino","São João da Lagoa",
    "São João da Ponte","São João das Missões","São João do Pacuí","São João do Paraíso","São Romão","Taiobeiras",
    "Teófilo Otoni","Três Marias","Turmalina","Ubaí","Umburatiba","Unaí","Uruana de Minas","Urucuia",
    "Vargem Grande do Rio Pardo","Varzelândia","Verdelândia","Veredinha","Virgem da Lapa","Várzea da Palma",
    "Água Boa","Águas Formosas","Águas Vermelhas"
]
df2021_reduzida = df2021.drop(columns=['TP_ESTADO_CIVIL','TP_NACIONALIDADE','TP_ANO_CONCLUIU','IN_TREINEIRO',
    'CO_MUNICIPIO_PROVA','NO_MUNICIPIO_PROVA','CO_UF_PROVA','SG_UF_PROVA',
    'CO_PROVA_CN','CO_PROVA_CH','CO_PROVA_LC','CO_PROVA_MT',
    'TX_RESPOSTAS_CN','TX_RESPOSTAS_CH','TX_RESPOSTAS_LC','TX_RESPOSTAS_MT',
    'TX_GABARITO_CN','TX_GABARITO_CH','TX_GABARITO_LC','TX_GABARITO_MT'])
df2021_reduzida = df2021_reduzida[df2021_reduzida['NO_MUNICIPIO_ESC'].isin(cidades_filtro)]
df2021_reduzida

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ESCOLA,TP_ENSINO,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
300,210053840619,2021,3,F,1,2,2,1.0,3103405.0,Araçuaí,...,A,A,A,B,A,A,E,A,B,B
3045,210054247525,2021,3,F,3,2,2,1.0,3157005.0,Salinas,...,B,A,A,B,B,B,D,A,B,B
3393,210051919359,2021,4,F,3,2,2,1.0,5208103.0,Formoso,...,A,A,A,B,A,A,B,A,A,B
4234,210054312631,2021,3,M,3,2,2,1.0,3125408.0,Felício dos Santos,...,A,A,A,B,A,A,C,A,A,A
4304,210052877241,2021,3,F,3,2,2,1.0,3170404.0,Unaí,...,A,A,A,B,A,A,D,A,B,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3367601,210052795497,2021,2,M,3,2,2,1.0,3113008.0,Caraí,...,A,A,A,B,A,A,D,A,A,B
3380593,210052212675,2021,2,F,5,2,2,1.0,3148509.0,Pavão,...,A,A,A,B,A,A,C,A,B,B
3381355,210053797477,2021,12,F,2,2,2,1.0,3113701.0,Carlos Chagas,...,A,A,A,B,A,A,C,A,A,B
3384332,210053447903,2021,3,F,1,2,2,1.0,3157005.0,Salinas,...,B,A,A,B,A,A,D,A,B,B


**Remoção e filtro dos microdados do ENEM 2022**

In [23]:
cidades_filtro = [
    "Almenara","Alvorada de Minas","Angelândia","Araçuaí","Aricanduva","Arinos","Ataléia",
    "Augusto de Lima","Bandeira","Berilo","Berizal","Bertópolis","Bocaiúva","Bonfinópolis de Minas",
    "Bonito de Minas","Botumirim","Brasília de Minas","Buenópolis","Buritis","Buritizeiro","Cabeceira Grande",
    "Cachoeira de Pajeú","Campanário","Campo Azul","Capelinha","Capitão Enéas","Caraí","Carbonita",
    "Carlos Chagas","Catuji","Catuti","Chapada Gaúcha","Chapada do Norte","Claro dos Poções","Comercinho",
    "Coração de Jesus","Corinto","Coronel Murta","Couto de Magalhães de Minas","Cristália","Crisólita",
    "Curral de Dentro","Curvelo","Cônego Marinho","Datas","Diamantina","Divisa Alegre","Divisópolis",
    "Dom Bosco","Engenheiro Navarro","Espinosa","Felisburgo","Felixlândia","Felício dos Santos","Formoso",
    "Francisco Badaró","Francisco Dumont","Francisco Sá","Franciscópolis","Frei Gaspar","Fronteira dos Vales",
    "Fruta de Leite","Gameleiras","Glaucilândia","Gouveia","Grão Mogol","Guaraciama","Ibiaí","Ibiracatu",
    "Icaraí de Minas","Indaiabira","Inimutaba","Itacambira","Itacarambi","Itaipé","Itamarandiba","Itambacuri",
    "Itaobim","Itinga","Jacinto","Janaúba","Januária","Japonvar","Jaíba","Jenipapo de Minas","Jequitaí",
    "Jequitinhonha","Joaquim Felício","Joaíma","Jordânia","Josenópolis","José Gonçalves de Minas","Juramento",
    "Juvenília","Ladainha","Lagoa dos Patos","Lassance","Leme do Prado","Lontra","Luislândia","Machacalis",
    "Malacacheta","Mamonas","Manga","Mata Verde","Matias Cardoso","Mato Verde","Medina","Minas Novas",
    "Mirabela","Miravânia","Monjolos","Montalvânia","Monte Azul","Monte Formoso","Montes Claros","Montezuma",
    "Morro da Garça","Nanuque","Natalândia","Ninheira","Nova Módica","Nova Porteirinha","Novo Cruzeiro",
    "Novo Oriente de Minas","Novorizonte","Olhos-d'Água","Ouro Verde de Minas","Padre Carvalho","Padre Paraíso",
    "Pai Pedro","Palmópolis","Patis","Pavão","Pedra Azul","Pedras de Maria da Cruz","Pescador","Pintópolis",
    "Pirapora","Ponto Chique","Ponto dos Volantes","Porteirinha","Poté","Presidente Juscelino",
    "Presidente Kubitschek","Riachinho","Riacho dos Machados","Rio Pardo de Minas","Rio do Prado","Rubelita",
    "Rubim","Salinas","Salto da Divisa","Santa Cruz de Salinas","Santa Fé de Minas","Santa Helena de Minas",
    "Santa Maria do Salto","Santo Antônio do Itambé","Santo Antônio do Jacinto","Santo Antônio do Retiro",
    "Santo Hipólito","Senador Modestino Gonçalves","Serra Azul de Minas","Serra dos Aimorés","Serranópolis de Minas",
    "Serro","Setubinha","São Francisco","São Gonçalo do Rio Preto","São José do Divino","São João da Lagoa",
    "São João da Ponte","São João das Missões","São João do Pacuí","São João do Paraíso","São Romão","Taiobeiras",
    "Teófilo Otoni","Três Marias","Turmalina","Ubaí","Umburatiba","Unaí","Uruana de Minas","Urucuia",
    "Vargem Grande do Rio Pardo","Varzelândia","Verdelândia","Veredinha","Virgem da Lapa","Várzea da Palma",
    "Água Boa","Águas Formosas","Águas Vermelhas"
]
df2022_reduzida = df2022.drop(columns=['TP_ESTADO_CIVIL','TP_NACIONALIDADE','TP_ANO_CONCLUIU','IN_TREINEIRO',
    'CO_MUNICIPIO_PROVA','NO_MUNICIPIO_PROVA','CO_UF_PROVA','SG_UF_PROVA',
    'CO_PROVA_CN','CO_PROVA_CH','CO_PROVA_LC','CO_PROVA_MT',
    'TX_RESPOSTAS_CN','TX_RESPOSTAS_CH','TX_RESPOSTAS_LC','TX_RESPOSTAS_MT',
    'TX_GABARITO_CN','TX_GABARITO_CH','TX_GABARITO_LC','TX_GABARITO_MT'])
df2022_reduzida = df2022_reduzida[df2022_reduzida['NO_MUNICIPIO_ESC'].isin(cidades_filtro)]
df2022_reduzida

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ESCOLA,TP_ENSINO,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
388,210056247191,2022,3,F,3,2,2,1.0,3143302.0,Montes Claros,...,A,A,A,B,A,B,C,A,B,B
823,210057917876,2022,3,F,3,2,2,1.0,2111052.0,São João do Paraíso,...,B,A,A,B,A,A,C,A,B,B
1076,210055813318,2022,2,F,3,2,2,1.0,3170404.0,Unaí,...,A,A,A,B,A,A,C,A,A,A
1211,210054558248,2022,3,F,1,2,2,1.0,3170404.0,Unaí,...,B,A,A,B,A,A,D,A,C,B
2032,210055740034,2022,2,M,1,2,2,1.0,3143302.0,Montes Claros,...,B,A,A,B,A,A,B,A,B,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3473582,210056592398,2022,2,F,2,2,2,1.0,3143302.0,Montes Claros,...,B,A,B,B,A,B,E,B,C,B
3474281,210055465294,2022,3,M,2,2,2,1.0,3101706.0,Almenara,...,B,A,A,B,A,A,C,A,A,B
3474320,210057847560,2022,2,F,2,2,2,1.0,3138104.0,Lassance,...,A,A,A,B,B,B,E,A,B,B
3474794,210055018400,2022,8,M,3,2,2,NaN,3121605.0,Diamantina,...,B,A,A,B,A,A,B,A,B,B


**Remoção e filtro dos microdados do ENEM 2023**

In [24]:
cidades_filtro = [
    "Almenara","Alvorada de Minas","Angelândia","Araçuaí","Aricanduva","Arinos","Ataléia",
    "Augusto de Lima","Bandeira","Berilo","Berizal","Bertópolis","Bocaiúva","Bonfinópolis de Minas",
    "Bonito de Minas","Botumirim","Brasília de Minas","Buenópolis","Buritis","Buritizeiro","Cabeceira Grande",
    "Cachoeira de Pajeú","Campanário","Campo Azul","Capelinha","Capitão Enéas","Caraí","Carbonita",
    "Carlos Chagas","Catuji","Catuti","Chapada Gaúcha","Chapada do Norte","Claro dos Poções","Comercinho",
    "Coração de Jesus","Corinto","Coronel Murta","Couto de Magalhães de Minas","Cristália","Crisólita",
    "Curral de Dentro","Curvelo","Cônego Marinho","Datas","Diamantina","Divisa Alegre","Divisópolis",
    "Dom Bosco","Engenheiro Navarro","Espinosa","Felisburgo","Felixlândia","Felício dos Santos","Formoso",
    "Francisco Badaró","Francisco Dumont","Francisco Sá","Franciscópolis","Frei Gaspar","Fronteira dos Vales",
    "Fruta de Leite","Gameleiras","Glaucilândia","Gouveia","Grão Mogol","Guaraciama","Ibiaí","Ibiracatu",
    "Icaraí de Minas","Indaiabira","Inimutaba","Itacambira","Itacarambi","Itaipé","Itamarandiba","Itambacuri",
    "Itaobim","Itinga","Jacinto","Janaúba","Januária","Japonvar","Jaíba","Jenipapo de Minas","Jequitaí",
    "Jequitinhonha","Joaquim Felício","Joaíma","Jordânia","Josenópolis","José Gonçalves de Minas","Juramento",
    "Juvenília","Ladainha","Lagoa dos Patos","Lassance","Leme do Prado","Lontra","Luislândia","Machacalis",
    "Malacacheta","Mamonas","Manga","Mata Verde","Matias Cardoso","Mato Verde","Medina","Minas Novas",
    "Mirabela","Miravânia","Monjolos","Montalvânia","Monte Azul","Monte Formoso","Montes Claros","Montezuma",
    "Morro da Garça","Nanuque","Natalândia","Ninheira","Nova Módica","Nova Porteirinha","Novo Cruzeiro",
    "Novo Oriente de Minas","Novorizonte","Olhos-d'Água","Ouro Verde de Minas","Padre Carvalho","Padre Paraíso",
    "Pai Pedro","Palmópolis","Patis","Pavão","Pedra Azul","Pedras de Maria da Cruz","Pescador","Pintópolis",
    "Pirapora","Ponto Chique","Ponto dos Volantes","Porteirinha","Poté","Presidente Juscelino",
    "Presidente Kubitschek","Riachinho","Riacho dos Machados","Rio Pardo de Minas","Rio do Prado","Rubelita",
    "Rubim","Salinas","Salto da Divisa","Santa Cruz de Salinas","Santa Fé de Minas","Santa Helena de Minas",
    "Santa Maria do Salto","Santo Antônio do Itambé","Santo Antônio do Jacinto","Santo Antônio do Retiro",
    "Santo Hipólito","Senador Modestino Gonçalves","Serra Azul de Minas","Serra dos Aimorés","Serranópolis de Minas",
    "Serro","Setubinha","São Francisco","São Gonçalo do Rio Preto","São José do Divino","São João da Lagoa",
    "São João da Ponte","São João das Missões","São João do Pacuí","São João do Paraíso","São Romão","Taiobeiras",
    "Teófilo Otoni","Três Marias","Turmalina","Ubaí","Umburatiba","Unaí","Uruana de Minas","Urucuia",
    "Vargem Grande do Rio Pardo","Varzelândia","Verdelândia","Veredinha","Virgem da Lapa","Várzea da Palma",
    "Água Boa","Águas Formosas","Águas Vermelhas"
]
df2023_reduzida = df2023.drop(columns=['TP_ESTADO_CIVIL','TP_NACIONALIDADE','TP_ANO_CONCLUIU','IN_TREINEIRO',
    'CO_MUNICIPIO_PROVA','NO_MUNICIPIO_PROVA','CO_UF_PROVA','SG_UF_PROVA',
    'CO_PROVA_CN','CO_PROVA_CH','CO_PROVA_LC','CO_PROVA_MT',
    'TX_RESPOSTAS_CN','TX_RESPOSTAS_CH','TX_RESPOSTAS_LC','TX_RESPOSTAS_MT',
    'TX_GABARITO_CN','TX_GABARITO_CH','TX_GABARITO_LC','TX_GABARITO_MT'])
df2023_reduzida = df2023_reduzida[df2023_reduzida['NO_MUNICIPIO_ESC'].isin(cidades_filtro)]
df2023_reduzida

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ESCOLA,TP_ENSINO,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
878,210061103984,2023,3,M,3,2,2,1.0,3135209.0,Januária,...,B,A,A,C,B,A,D,A,B,B
3721,210058118676,2023,3,F,2,2,2,1.0,3135456.0,Jenipapo de Minas,...,A,A,A,B,A,A,C,A,A,B
5546,210061420420,2023,3,F,4,2,2,1.0,3142007.0,Mirabela,...,A,A,A,A,A,A,B,A,A,A
6082,210059483609,2023,3,F,3,2,2,1.0,3148707.0,Pedra Azul,...,B,A,A,B,A,A,C,A,B,B
6296,210060079377,2023,3,F,3,2,2,1.0,3167103.0,Serro,...,B,A,A,B,A,A,B,A,A,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3930550,210061556296,2023,3,F,3,2,2,1.0,3143302.0,Montes Claros,...,A,A,A,B,A,A,B,A,A,B
3931291,210059204619,2023,4,M,1,2,2,NaN,3168606.0,Teófilo Otoni,...,B,A,A,A,A,A,D,A,A,B
3931834,210061038879,2023,2,M,3,2,2,1.0,3143302.0,Montes Claros,...,B,A,B,B,B,A,E,B,C,B
3933287,210061615619,2023,3,M,3,2,2,1.0,3103405.0,Araçuaí,...,B,A,A,B,A,A,C,A,B,B


**Concatenando as bases de todos os anos**

In [25]:
df_combinado = pd.concat([df2018_reduzida, df2019_reduzida, df2020_reduzida, df2021_reduzida, df2022_reduzida, df2023_reduzida], ignore_index=True)
df_combinado.to_csv(r"C:\Users\fernanda.leal\Documents\FERNANDA\TCC\DESENVOLVIMENTO\BASES\ENEM_CONCAT.csv", index=False, encoding="utf-8-sig")
df_combinado

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ESCOLA,TP_ENSINO,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,180007186628,2018,3,M,0,2,2,1.0,3155603.0,Rio Pardo de Minas,...,A,A,A,B,B,A,D,A,A,A
1,180007186632,2018,2,F,3,2,2,1.0,3161106.0,São Francisco,...,A,A,A,B,B,A,D,A,A,B
2,180007186772,2018,2,M,3,2,2,1.0,3142700.0,Montalvânia,...,A,A,A,B,A,A,C,A,B,B
3,180007186871,2018,2,M,3,2,2,1.0,3116100.0,Chapada do Norte,...,B,A,A,C,B,A,C,A,B,B
4,180007186944,2018,3,F,1,2,2,1.0,3168606.0,Teófilo Otoni,...,B,A,A,B,A,B,E,A,B,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75546,210061556296,2023,3,F,3,2,2,1.0,3143302.0,Montes Claros,...,A,A,A,B,A,A,B,A,A,B
75547,210059204619,2023,4,M,1,2,2,NaN,3168606.0,Teófilo Otoni,...,B,A,A,A,A,A,D,A,A,B
75548,210061038879,2023,2,M,3,2,2,1.0,3143302.0,Montes Claros,...,B,A,B,B,B,A,E,B,C,B
75549,210061615619,2023,3,M,3,2,2,1.0,3103405.0,Araçuaí,...,B,A,A,B,A,A,C,A,B,B


**Renomeando as colunas do questionários para melhor visualização**

In [26]:
rename_dict = {
    "Q001": "ESQ_PAI",
    "Q002": "ESC_MAE",
    "Q003": "OCU_PAI",
    "Q004": "OCU_MAE",
    "Q005": "QTD_PESSOAS",
    "Q006": "RENDA_MENSAL",
    "Q007": "EMP_DOMESTICO",
    "Q008": "BANHEIRO",
    "Q009": "QTD_QUARTOS",
    "Q010": "CARRO",
    "Q011": "MOTO",
    "Q012": "GELADEIRA",
    "Q013": "FREEZER",
    "Q014": "LAVA_ROUPA",
    "Q015": "SECA_ROUPA",
    "Q016": "MICROONDAS",
    "Q017": "LAVA_LOUCA",
    "Q018": "ASPIRADOR",
    "Q019": "TV",
    "Q020": "DVD",
    "Q021": "TV_ASSINATURA",
    "Q022": "CELULAR",
    "Q023": "TELEFONE_FIXO",
    "Q024": "COMPUTADOR",
    "Q025": "INTERNET"
}

df_renomeado = df_combinado.rename(columns=rename_dict)
df_renomeado.to_csv(r"C:\Users\fernanda.leal\Documents\FERNANDA\TCC\DESENVOLVIMENTO\BASES\ENEM_RENOMEADO.csv", index=False, encoding="utf-8-sig")
df_renomeado


,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ESCOLA,TP_ENSINO,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,...,MICROONDAS,LAVA_LOUCA,ASPIRADOR,TV,DVD,TV_ASSINATURA,CELULAR,TELEFONE_FIXO,COMPUTADOR,INTERNET
0,180007186628,2018,3,M,0,2,2,1.0,3155603.0,Rio Pardo de Minas,...,A,A,A,B,B,A,D,A,A,A
1,180007186632,2018,2,F,3,2,2,1.0,3161106.0,São Francisco,...,A,A,A,B,B,A,D,A,A,B
2,180007186772,2018,2,M,3,2,2,1.0,3142700.0,Montalvânia,...,A,A,A,B,A,A,C,A,B,B
3,180007186871,2018,2,M,3,2,2,1.0,3116100.0,Chapada do Norte,...,B,A,A,C,B,A,C,A,B,B
4,180007186944,2018,3,F,1,2,2,1.0,3168606.0,Teófilo Otoni,...,B,A,A,B,A,B,E,A,B,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75546,210061556296,2023,3,F,3,2,2,1.0,3143302.0,Montes Claros,...,A,A,A,B,A,A,B,A,A,B
75547,210059204619,2023,4,M,1,2,2,NaN,3168606.0,Teófilo Otoni,...,B,A,A,A,A,A,D,A,A,B
75548,210061038879,2023,2,M,3,2,2,1.0,3143302.0,Montes Claros,...,B,A,B,B,B,A,E,B,C,B
75549,210061615619,2023,3,M,3,2,2,1.0,3103405.0,Araçuaí,...,B,A,A,B,A,A,C,A,B,B
